# Initialize your account

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
API = "6414ab61cb1a21db7fa9b89369a38489cb597ac7c6fc85e04046964fcff9fae57c6e1ca104fe2d0f41ef9bed371f6eff1c31e01e6fd5ebeb2a412c2bb0916a65"
service = QiskitRuntimeService(channel = "ibm_quantum" ,token=API)

service.backends()

In [ ]:
service.backends(simulator=False, operational=True, min_num_qubits=5)

In [ ]:
backend = service.least_busy(operational=True, min_num_qubits=5)

In [ ]:
# backend = service.backend("ibm_kyoto")
 
print(
    f"Name: {backend.name}\n"
    f"Version: {backend.version}\n"
    f"No. of qubits: {backend.num_qubits}\n"
)

# VQE Starts Here

## Setup your Hamiltonian

In [ ]:
# padding_str = (backend.configuration().n_qubits-2) * 'I'
# len(padding_str)

In [ ]:
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

In [ ]:
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II"+padding_str, -1.052373245772859),
        ("IZ"+padding_str, 0.39793742484318045),
        ("ZI"+padding_str, -0.39793742484318045),
        ("ZZ"+padding_str, -0.01128010425623538),
        ("XX"+padding_str, 0.18093119978423156),
    ]
)

In [ ]:
from qiskit_algorithms.optimizers import SPSA
from qiskit.circuit.library import EfficientSU2
from qiskit.primitives import Estimator
import numpy as np

estimator = Estimator()
ansatz = EfficientSU2(2)
ansatz.decompose().draw('mpl')

### Transpile the circuit according to Backend Topology

In [ ]:
callback = lambda nfev, params, fval, step, acc: print(f'Evaluation {nfev}: {fval}')
optimizer = SPSA(callback=callback, maxiter=1000)

In [ ]:
cost_func = lambda params: estimator.run(ansatz, H2_op, parameter_values=params).result().values[0]
result = optimizer.minimize(cost_func, x0=np.zeros(ansatz.num_parameters))

In [ ]:
print(result)

# Run VQE Circuit in backend

In [ ]:
padding_str = (backend.configuration().n_qubits-H2_op.num_qubits) * 'I'
len(padding_str)

In [ ]:
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II"+padding_str, -1.052373245772859),
        ("IZ"+padding_str, 0.39793742484318045),
        ("ZI"+padding_str, -0.39793742484318045),
        ("ZZ"+padding_str, -0.01128010425623538),
        ("XX"+padding_str, 0.18093119978423156),
    ]
)

In [ ]:
H2_op.num_qubits

In [ ]:
from qiskit_algorithms.optimizers import SPSA
from qiskit.circuit.library import EfficientSU2
from qiskit.primitives import Estimator
import numpy as np

estimator = Estimator()
ansatz = EfficientSU2(2)

# Transpile the circuit

In [ ]:
from qiskit import transpile
# ansatz.decompose().draw('mpl')
transpiled = transpile(ansatz.decompose(), backend=backend)
transpiled.draw('mpl')

In [ ]:
from qiskit_ibm_runtime import Options

options = Options()
options.execution.init_qubits = True
options.execution.shots = 5000

In [ ]:
options.environment.callback = []
options.environment.job_tabs = []
options.environment.log_level = 'DEBUG'

In [ ]:
from qiskit_ibm_runtime import Estimator, Session

with Session(service=service, backend=backend) as session:
    estimator = Estimator(options=options)
    cost_func = lambda params: estimator.run(transpiled, H2_op, parameter_values=params).result().values[0]
    result = optimizer.minimize(cost_func, x0=np.zeros(ansatz.num_parameters))

In [ ]:
print(result)